<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#p-values-calculation" data-toc-modified-id="p-values-calculation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>p-values calculation</a></span></li></ul></div>

# Import libraries

In [14]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy as np
from scipy import stats
import shapely.geometry as sgeom
import scipy as sp
%matplotlib inline

In [15]:
%reload_ext watermark
%watermark --iversions -v -m -p scipy, shapely, matplotlib

xarray 0.14.1
pandas 1.0.1
numpy  1.17.3
scipy  1.2.1
CPython 3.6.9
IPython 7.1.1

compiler   : GCC 8.4.0
system     : Linux
release    : 4.15.0-96-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


# Load data

In [2]:
var = 'accelogw'
root_path = '/mnt/4data/CMAM/0A.daily/'

In [3]:
ds = xr.open_mfdataset(f'{root_path}{var}/{var}_6hrPlev_CMAM_CMAM30-SD_r1i1p1_*010100-*123118.nc', \
                      concat_dim = 'time', parallel = True, combine = 'nested')#.load()

In [4]:
ds_clim = xr.open_dataset(f'{root_path}{var}/{var}_climatology_woSSW.nc')

In [6]:
ds_anom = (ds[var].groupby('time.month') - ds_clim[var]).to_dataset()
ds_anom

<xarray.Dataset>
Dimensions:   (lat: 48, lon: 96, plev: 63, time: 46720)
Coordinates:
  * lat       (lat) float64 -87.16 -83.48 -79.78 -76.07 ... 79.78 83.48 87.16
  * lon       (lon) float64 0.0 3.75 7.5 11.25 15.0 ... 345.0 348.8 352.5 356.2
  * plev      (plev) float64 1e+05 9.25e+04 8.5e+04 8e+04 ... 0.2 0.15 0.1 0.07
  * time      (time) object 1979-01-01 00:00:00 ... 2010-12-31 18:00:00
    month     (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    accelogw  (time, plev, lat, lon) float64 dask.array<chunksize=(124, 63, 48, 96), meta=np.ndarray>

In [7]:
what = 'anomalies'
outfile = f'{root_path}{var}/{var}_6hrPlev_CMAM_CMAM30-SD_r1i1p1_19790101-20101231_{what}.zarr'
ds_anom.chunk({'time': 124}).to_zarr(outfile, \
                                     mode = 'w')

# p-values calculation

In [3]:
def g_kde(y, x):
    kde = stats.gaussian_kde(y)
    return kde(x)

In [4]:
time_scale = '20' # 20 or 30
what = 'anomalies'
factor = 24*3600
comp_name_ls = ['himalayas', 'eastasia', 'westamer',]
line_width = 5
its = 10000
ts_anom = xr.open_zarr(outfile)
ts_anom

<xarray.Dataset>
Dimensions:   (lat: 48, lon: 96, plev: 63, time: 46720)
Coordinates:
  * lat       (lat) float64 -87.16 -83.48 -79.78 -76.07 ... 79.78 83.48 87.16
  * lon       (lon) float64 0.0 3.75 7.5 11.25 15.0 ... 345.0 348.8 352.5 356.2
    month     (time) int64 dask.array<chunksize=(124,), meta=np.ndarray>
  * plev      (plev) float64 1e+05 9.25e+04 8.5e+04 8e+04 ... 0.2 0.15 0.1 0.07
  * time      (time) object 1979-01-01 00:00:00 ... 2010-12-31 18:00:00
Data variables:
    accelogw  (time, plev, lat, lon) float64 dask.array<chunksize=(124, 63, 48, 96), meta=np.ndarray>

In [10]:
DJF_bool = 'DJF'
w_clim = f'_{DJF_bool}only'
infile = f'{root_path}composites_woSSW'

if DJF_bool == 'DJF':                                                          
    size_dict = {'20': [37,37,25], '30': []}                               
else:
    size_dict = {'20': [45,74,36], '30': [38,66,35]} 

for comp_name, size in zip(comp_name_ls, size_dict[time_scale]):
    print(comp_name, size)

    if comp_name == 'himalayas':
        box = sgeom.box(minx=70, maxx=102.5, miny=20, maxy=40)
    elif comp_name == 'eastasia':
        box = sgeom.box(minx=110, maxx=145, miny=30, maxy=48)
    elif comp_name == 'westamer':
        box = sgeom.box(minx=-125, maxx=-102.5, miny=27.5, maxy=52)
    
    box_lats = np.array(box.bounds)[1::2]
    box_lons = np.array(box.bounds)[::2]
    box_lons[box_lons < 0] += 360
        
    sel_dict = dict(lat = slice(box_lats[0], box_lats[1]), \
                    lon = slice(box_lons[0], box_lons[1]))  
        
    ds_comp = xr.open_dataarray(f'{infile}{w_clim}/{var}_anomalies_comp_{comp_name}_20days.nc')            
    ds_comp = ds_comp.sel(**sel_dict).mean(['lat', 'lon'])*factor # .load()
    ts_sel_anom = ts_anom[var].sel(**sel_dict).mean(['lat', 'lon'])*factor # .load()
    print("".ljust(line_width)+' files prepared')
    
    rnd_means = xr.concat([ts_sel_anom.isel(time = random.sample(range(ts_sel_anom.time.shape[0]), size)).mean('time') \
                       for n in range(its)], dim = 'its')
    print("".ljust(line_width)+'{} samples generated'.format(its))

    da_kde = xr.apply_ufunc(g_kde, rnd_means, ds_comp,\
                       input_core_dims=[['its'], []],\
                       vectorize=True, dask='allowed')
    print("".ljust(line_width)+'p-values calculated')

    outfile_name = f'{infile}{w_clim}/{var}_pvalues_from{its}_comp_{comp_name}_{time_scale}days_latlonbox.nc' 
    da_kde.to_netcdf(outfile_name)
    print("".ljust(line_width)+outfile_name+" saved")
    print()
    #sys.exit()
                                
print('Done')

himalayas 37
      files prepared
     10000 samples generated
     p-values calculated
     /mnt/4data/CMAM/0A.daily/composites_woSSW_DJFonly/accelogw_pvalues_from10000_comp_himalayas_20days_latlonbox.nc saved

eastasia 37
      files prepared
     10000 samples generated
     p-values calculated
     /mnt/4data/CMAM/0A.daily/composites_woSSW_DJFonly/accelogw_pvalues_from10000_comp_eastasia_20days_latlonbox.nc saved

westamer 25
      files prepared
     10000 samples generated
     p-values calculated
     /mnt/4data/CMAM/0A.daily/composites_woSSW_DJFonly/accelogw_pvalues_from10000_comp_westamer_20days_latlonbox.nc saved

Done
